# HSE 2023: Введение в машинное обучение БИ 23/24

## ДЗ 2

# Внимание!
Если в задании просят объяснить что-либо, то это значит, что требуется письменный ответ, который является частью задания и оценивается

Мы только принимаем ipynb ноутбуки. Если вы используете Google Colab, то вам необходимо скачать ноутбук перед сдачей ДЗ


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
# from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from math import sqrt
import random
import sys

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

### Данные

Для этого ДЗ мы будем использовать датасет треков со стримингового сервиса Spotify

**Описание данных**

- **track_id:** The Spotify ID for the track
- **artists:** The artists' names who performed the track. If there is more than one artist, they are separated by a ;
- **album_name:** The album name in which the track appears
- **track_name:** Name of the track
- **popularity:** The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.
- **duration_ms:** The track length in milliseconds
- **explicit:** Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)
- **danceability:** Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable
- **key:** The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1
- **loudness:** The overall loudness of a track in decibels (dB)
- **mode:** Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0
- **speechiness:** Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks
- **acousticness:** A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic
- **instrumentalness:** Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content
- **liveness:** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live
- **valence:** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)
- **tempo:** The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration
- **time_signature:** An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.
- **track_genre:** The genre in which the track belongs

**Целевая переменная**
- **energy:** Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale

In [2]:
data = pd.read_csv('dataset.csv')

y = data['energy']
X = data.drop(['energy'], axis=1)
columns = X.columns

In [3]:
data

,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


## Линейная регрессия

#### 0. [0.25 балла] Закодируйте категориальные признаки. Объясните выбранный вами метод.

In [4]:
category_method = {"explicit": "one-hot", "key": "one-hot", "time_signature": "one-hot", "track_genre": "one-hot", "mode": "one-hot"}
drop_columns = ["artists", "album_name", "track_name"]

In [5]:
for category in category_method.keys():
    print(f"category: {category} | nunique: {data[category].nunique()}")

category: explicit | nunique: 2
category: key | nunique: 12
category: time_signature | nunique: 5
category: track_genre | nunique: 114
category: mode | nunique: 2


In [6]:
data_dropped = data.drop(drop_columns, axis=1)
data_without_categorical = data_dropped.drop(list(category_method), axis=1)
data_encoded = pd.get_dummies(data_dropped, columns=list(category_method), drop_first=True)

In [7]:
data_without_categorical.columns

Index(['popularity', 'duration_ms', 'danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo'],
      dtype='object')

One-hot кодирование было выбрано потому, что среди категориальных переменных не много уникальных значений. Дроп колонок с именами сделан потому, что они в большинстве своем уникальны и не помогают при прогнозе

#### 1. [0.25 балла] Разбейте данные на train и test с пропорцией 75:25 и random_state=7.

In [8]:
from sklearn.model_selection import train_test_split

y = data_without_categorical["energy"]
X = data_without_categorical.drop("energy", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

#### 2. [0.75 балла] Обучите модели на train'е, исключив категориальные признаки, используя библиотеку StatsModels и примените ее к test'у; используйте $RMSE$ и $R ^ 2$ в качестве метрики качества. Попробуйте также применить реализации линейной регрессии из sklearn:

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.03$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.05$
* [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) with $\alpha = 0.01$, $l_{1}$_$ratio = 0.4$

Не забывайте скейлить данные с помощью StandardScaler перед обучением моделей!

In [9]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

In [10]:
X_train_const = sm.add_constant(X_train_scaled)

stats_models = {
    "LinearReggression": sm.OLS(y_train, X_train_const),
    "Ridge": sm.OLS(y_train, X_train_const),
    "Lasso": sm.OLS(y_train, X_train_const),
    "ElasticNet": sm.OLS(y_train, X_train_const)
}

stats_models["LinearReggression"] = stats_models["LinearReggression"].fit()
stats_models["Ridge"] = stats_models["Ridge"].fit_regularized(alpha=0.03, L1_wt=0)
stats_models["Lasso"] = stats_models["Lasso"].fit_regularized(alpha=0.05, L1_wt=1)
stats_models["ElasticNet"] = stats_models["ElasticNet"].fit_regularized(alpha=0.01, L1_wt=0.4)

In [11]:
X_train_const

,const,popularity,duration_ms,danceability,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
7124,1.0,-0.146253,-0.131016,-0.560354,0.303882,-0.471616,-0.049361,-0.503763,0.273146,1.359439,2.160351
101347,1.0,0.301625,-1.300885,-2.780935,-2.306411,0.675453,-0.945174,2.017181,0.708404,-1.829734,-1.443998
70140,1.0,0.973440,0.055443,0.930393,-0.094352,-0.508526,-0.900692,-0.499874,1.421599,-0.428197,0.059538
73606,1.0,0.435988,2.314320,0.746208,-0.758010,-0.419562,-0.473222,2.363055,-0.445292,-1.691125,-0.137525
38529,1.0,-0.235828,-0.759591,-0.341634,0.928968,-0.375080,-0.750687,-0.504128,0.451445,0.498439,0.061733
...,...,...,...,...,...,...,...,...,...,...,...
104003,1.0,-1.489884,-0.431347,-0.497041,0.842482,-0.411044,-0.437149,-0.504141,-0.748924,-0.084569,-1.073618
10742,1.0,-0.997219,1.712364,-1.924475,0.701718,-0.169706,-0.943550,-0.368701,-0.606286,0.614269,1.388694
49689,1.0,-0.549342,0.481916,-1.015061,0.451604,-0.331545,-0.925823,-0.435290,1.495016,-1.655990,0.795475
58564,1.0,-0.459767,-0.065798,0.205165,0.878667,-0.176331,-0.945092,2.550538,-0.640897,0.374888,0.759421


In [12]:
X_test_const = sm.add_constant(X_test_scaled)

for model_name, model in stats_models.items():
    y_pred = model.predict(X_test_const)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} | RMSE: {rmse} | R^2: {r2}")

LinearReggression | RMSE: 0.12187052768508619 | R^2: 0.7636016854471843
Ridge | RMSE: 0.12329725300411724 | R^2: 0.7580343068334253
Lasso | RMSE: 0.154930535710214 | R^2: 0.6179492665266193
ElasticNet | RMSE: 0.12280468241939398 | R^2: 0.7599637433553765


In [13]:
sklearn_models = {
    "LinearReggression": LinearRegression(),
    "Ridge": Ridge(0.03),
    "Lasso": Lasso(0.05),
    "ElasticNet": ElasticNet(0.01, l1_ratio=0.4)
}

for model in sklearn_models.values():
    model.fit(X_train_scaled, y_train)

In [14]:
for model_name, model in sklearn_models.items():
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} | RMSE: {rmse} | R^2: {r2}")

LinearReggression | RMSE: 0.12187052768508617 | R^2: 0.7636016854471843
Ridge | RMSE: 0.12187052713188165 | R^2: 0.7636016875933409
Lasso | RMSE: 0.1471071204791906 | R^2: 0.6555593645544675
ElasticNet | RMSE: 0.12259881682500326 | R^2: 0.7607678460441432


#### 3. [0.25 балла] Повторите шаги из предыдущего пункта, добавив категориальные признаки. Прокомментируйте изменения значений метрик качества

In [15]:
y = data_encoded["energy"]
X = data_encoded.drop("energy", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [16]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

In [17]:
X_train_const = sm.add_constant(X_train_scaled)

stats_models_cat = {
    "LinearReggression": sm.OLS(y_train, X_train_const),
    "Ridge": sm.OLS(y_train, X_train_const),
    "Lasso": sm.OLS(y_train, X_train_const),
    "ElasticNet": sm.OLS(y_train, X_train_const)
}

stats_models_cat["LinearReggression"] = stats_models_cat["LinearReggression"].fit()
stats_models_cat["Ridge"] = stats_models_cat["Ridge"].fit_regularized(alpha=0.03, L1_wt=0)
stats_models_cat["Lasso"] = stats_models_cat["Lasso"].fit_regularized(alpha=0.05, L1_wt=1)
stats_models_cat["ElasticNet"] = stats_models_cat["ElasticNet"].fit_regularized(alpha=0.01, L1_wt=0.4)

In [18]:
X_test_const = sm.add_constant(X_test_scaled)

for model_name, model in stats_models_cat.items():
    y_pred = model.predict(X_test_const)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} | RMSE: {rmse} | R^2: {r2}")

LinearReggression | RMSE: 0.11186322274286671 | R^2: 0.8008310440515642
Ridge | RMSE: 0.11401910506129356 | R^2: 0.7930801054266848
Lasso | RMSE: 0.154930535710214 | R^2: 0.6179492665266193
ElasticNet | RMSE: 0.11849719098409722 | R^2: 0.7765074228941483


In [19]:
sklearn_models_cat = {
    "LinearReggression": LinearRegression(),
    "Ridge": Ridge(0.03),
    "Lasso": Lasso(0.05),
    "ElasticNet": ElasticNet(0.01, l1_ratio=0.4)
}

for model in sklearn_models_cat.values():
    model.fit(X_train_scaled, y_train)

In [20]:
for model_name, model in sklearn_models_cat.items():
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} | RMSE: {rmse} | R^2: {r2}")

LinearReggression | RMSE: 0.11186322274286671 | R^2: 0.8008310440515642
Ridge | RMSE: 0.1118632295711286 | R^2: 0.8008310197365517
Lasso | RMSE: 0.1471071204791906 | R^2: 0.6555593645544675
ElasticNet | RMSE: 0.11731377177911514 | R^2: 0.7809491267271149


Stats без категориальных
```
LinearReggression | RMSE: 0.12187052768508619 | R^2: 0.7636016854471843
Ridge | RMSE: 0.12329725300411724 | R^2: 0.7580343068334253
Lasso | RMSE: 0.154930535710214 | R^2: 0.6179492665266193
ElasticNet | RMSE: 0.12280468241939398 | R^2: 0.7599637433553765
```

Sklearn без категориальных:
```
LinearReggression | RMSE: 0.12187052768508617 | R^2: 0.7636016854471843
Ridge | RMSE: 0.12187052713188165 | R^2: 0.7636016875933409
Lasso | RMSE: 0.1471071204791906 | R^2: 0.6555593645544675
ElasticNet | RMSE: 0.12259881682500326 | R^2: 0.7607678460441432
```

Stats с категориальными
```
LinearReggression | RMSE: 0.11186322274286671 | R^2: 0.8008310440515642
Ridge | RMSE: 0.11401910506129356 | R^2: 0.7930801054266848
Lasso | RMSE: 0.154930535710214 | R^2: 0.6179492665266193
ElasticNet | RMSE: 0.11849719098409722 | R^2: 0.7765074228941483
```

Sklearn с категориальными:
```
LinearReggression | RMSE: 0.11186322274286671 | R^2: 0.8008310440515642
Ridge | RMSE: 0.1118632295711286 | R^2: 0.8008310197365517
Lasso | RMSE: 0.1471071204791906 | R^2: 0.6555593645544675
ElasticNet | RMSE: 0.11731377177911514 | R^2: 0.7809491267271149
```

Можно сделать следующий вывод:
Во всех моделях кроме Lasso результат с добавлением закодированных категориальных переменных улучшился. Более того, модели sklearn в целом справляются с прогнозом чуть лучше statsmodels.

#### 4. [1 балл] Исследуйте значения параметров полученных моделей и проверьте какие веса получились нулевыми. Прокомментируйте значимость коэффициентов, обшую значимость модели и остальные факторы из результирующей таблицы

In [21]:
stats_models["LinearReggression"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 energy   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                 2.785e+04
Date:                Mon, 23 Oct 2023   Prob (F-statistic):               0.00
Time:                        19:30:47   Log-Likelihood:                 58524.
No. Observations:               85500   AIC:                        -1.170e+05
Df Residuals:                   85489   BIC:                        -1.169e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.6420      0.000   1538.157      0.000       0.641       0.643
popularity          -0.0020      0.000     -4.787      0.000      -0.003      -0.001
duration_ms          0.0052      0.000     12.119      0.000       0.004       0.006
danceability        -0.0315      0.001    -62.887      0.000      -0.032      -0.030
loudness             0.1357      0.001    227.454      0.000       0.135       0.137
speechiness          0.0274      0.000     63.259      0.000       0.027       0.028
acousticness        -0.1069      0.001   -199.787      0.000      -0.108      -0.106
instrumentalness     0.0354      0.000     72.079      0.000       0.034       0.036
liveness             0.0258      0.000     59.100      0.000       0.025       0.027
valence              0.0413      0.001     81.585      0.000       0.040       0.042
tempo                0.0075      0.000     17.262      0.000       0.007       0.008
==============================================================================
Omnibus:                     4801.534   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19948.192
Skew:                           0.068   Prob(JB):                         0.00
Kurtosis:                       5.362   Cond. No.                         2.69
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

В данном случае все значения коэффициентов оказались ненулевыми. Более того при уровне значимости $\alpha = 0.05$ все коэффициенты являются значимыми. Так же согласно полю Prob (F-statistic) = 0.00 можно сделать вывод, что сама модель регрессии значима при уровне $\alpha = 0.05$. $R^2$ ближе к 1, чем к 0 -- это хорошо

In [22]:
stats_models_cat["LinearReggression"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 energy   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     2496.
Date:                Mon, 23 Oct 2023   Prob (F-statistic):               0.00
Time:                        19:30:47   Log-Likelihood:                 66185.
No. Observations:               85500   AIC:                        -1.321e+05
Df Residuals:                   85359   BIC:                        -1.308e+05
Df Model:                         140                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                             0.6420      0.000   1681.062      0.000       0.641       0.643
popularity                       -0.0032      0.000     -7.182      0.000      -0.004      -0.002
duration_ms                       0.0010      0.000      2.430      0.015       0.000       0.002
danceability                     -0.0219      0.001    -38.212      0.000      -0.023      -0.021
loudness                          0.1353      0.001    222.671      0.000       0.134       0.136
speechiness                       0.0224      0.001     42.230      0.000       0.021       0.023
acousticness                     -0.0825      0.001   -136.857      0.000      -0.084      -0.081
instrumentalness                  0.0240      0.001     43.681      0.000       0.023       0.025
liveness                          0.0203      0.000     47.873      0.000       0.019       0.021
valence                           0.0410      0.001     79.093      0.000       0.040       0.042
tempo                             0.0060      0.000     14.368      0.000       0.005       0.007
explicit_True                    -0.0025      0.000     -5.891      0.000      -0.003      -0.002
key_1                             0.0033      0.000      6.639      0.000       0.002       0.004
key_2                             0.0009      0.001      1.821      0.069   -6.93e-05       0.002
key_3                         -4.252e-05      0.000     -0.100      0.921      -0.001       0.001
key_4                             0.0007      0.000      1.441      0.150      -0.000       0.002
key_5                            -0.0010      0.000     -1.972      0.049      -0.002   -5.79e-06
key_6                             0.0012      0.000      2.534      0.011       0.000       0.002
key_7                             0.0009      0.001      1.755      0.079      -0.000       0.002
key_8                             0.0012      0.000      2.587      0.010       0.000       0.002
key_9                             0.0013      0.001      2.641      0.008       0.000       0.002
key_10                            0.0025      0.000      5.329      0.000       0.002       0.003
key_11                            0.0023      0.000      4.779      0.000       0.001       0.003
time_signature_1                  0.0261      0.001     25.481      0.000       0.024       0.028
time_signature_3                  0.0738      0.003     25.720      0.000       0.068       0.079
time_signature_4                  0.0934      0.003     28.833      0.000       0.087       0.100
time_signature_5                  0.0333      0.001     24.688      0.000       0.031       0.036
track_genre_afrobeat              0.0083      0.001     15.064      0.000       0.007       0.009
track_genre_alt-rock              0.0088     

In [23]:
model_params = stats_models_cat["LinearReggression"].params
model_pvalues = stats_models_cat["LinearReggression"].pvalues

In [24]:
model_params[abs(model_params) < 1E-6]

Series([], dtype: float64)

In [25]:
model_pvalues[abs(model_pvalues) > 0.05]

key_2                     0.068549
key_3                     0.920559
key_4                     0.149509
key_7                     0.079212
track_genre_children      0.542699
track_genre_chill         0.760872
track_genre_honky-tonk    0.558227
track_genre_show-tunes    0.292337
track_genre_songwriter    0.212717
track_genre_soul          0.058169
dtype: float64

С закодированными категориальными признаками тоже не получили нулевых коэффициентов, однако 10 переменных стали незначимыми при $\alpha = 0.05$. Prob (F-statistic):	0.00 ⇒ модель также значима. $R^2$ вырос, что тоже хорошо  

Summary имплементирован только у LinearReggression из statsmodels, поэтому для всех остальных моделей посмотрим значений получившихся коэффициентов. $R^2$ у всех них не ниже 0.6, что в целом является неплохим результатом

In [26]:
i, coefs = sklearn_models["LinearReggression"].intercept_, sklearn_models["LinearReggression"].coef_
print(i, coefs)

0.6420033217438598 [-0.00201958  0.00518748 -0.03145458  0.13568212  0.02743261 -0.1069128
  0.03540285  0.02583874  0.04126599  0.00751454]


In [27]:
i, coefs = sklearn_models_cat["LinearReggression"].intercept_, sklearn_models_cat["LinearReggression"].coef_
print(i, coefs[abs(coefs) < 1E-6])

0.6420033217438598 []


У модели из sklearn результат сильно похож на statsmodels. Только результат по $R^2$ получше

In [28]:
stats_models["Ridge"].params

array([ 0.6233042 , -0.00225141,  0.00534436, -0.02847249,  0.13049524,
        0.02637422, -0.10588113,  0.03185145,  0.02585905,  0.03902873,
        0.00874872])

In [29]:
stats_models_cat["Ridge"].params
model_params[abs(model_params) < 1E-6]

Series([], dtype: float64)

Нулевых коэффициентов нет у Ridge регрессии из statsmodels

In [30]:
i, coefs = sklearn_models["Ridge"].intercept_, sklearn_models["Ridge"].coef_
print(i, coefs)

0.6420033217438598 [-0.00201959  0.00518749 -0.03145455  0.13568205  0.0274326  -0.10691279
  0.03540281  0.02583874  0.04126596  0.00751456]


In [31]:
i, coefs = sklearn_models_cat["Ridge"].intercept_, sklearn_models_cat["Ridge"].coef_
print(i, coefs[abs(coefs) < 1E-6])

0.6420033217438598 []


Аналогично у моделей из sklearn

In [32]:
stats_models["Lasso"].params

const               0.592003
popularity          0.000000
duration_ms         0.000000
danceability        0.000000
loudness            0.095527
speechiness         0.000000
acousticness       -0.078565
instrumentalness    0.000000
liveness            0.000000
valence             0.000000
tempo               0.000000
dtype: float64

In [33]:
stats_models_cat["Lasso"].params
model_params[abs(model_params) < 1E-6]

Series([], dtype: float64)

У Lasso появляются нулевые веса. Это связано с регуляризацией

In [34]:
i, coefs = sklearn_models["Lasso"].intercept_, sklearn_models["Lasso"].coef_
print(i, coefs)

0.6420033217438597 [-0.          0.         -0.          0.09552992  0.         -0.0785626
  0.          0.          0.          0.        ]


In [35]:
i, coefs = sklearn_models_cat["Lasso"].intercept_, sklearn_models_cat["Lasso"].coef_
print(i, coefs[abs(coefs) < 1E-6])

0.6420033217438597 [-0.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0.  0.  0.
  0.  0. -0. -0.  0. -0.  0.  0. -0. -0.  0.  0. -0. -0. -0.  0. -0. -0.
  0. -0. -0. -0.  0.  0. -0. -0. -0.  0.  0.  0.  0. -0.  0. -0.  0.  0.
 -0. -0. -0. -0.  0. -0. -0.  0. -0. -0.  0.  0.  0.  0. -0.  0.  0.  0.
  0.  0. -0. -0.  0.  0. -0. -0. -0.  0.  0.  0.  0. -0.  0. -0. -0. -0.
 -0. -0. -0. -0.  0.  0.  0. -0. -0. -0.  0.  0. -0. -0. -0.  0.  0. -0.
  0.  0.  0. -0. -0. -0. -0.  0. -0. -0.  0.  0.  0. -0. -0.  0.  0. -0.
 -0.  0. -0. -0.  0. -0.  0.  0. -0. -0. -0. -0.]


У модели sklearn нулевые веса есть не только на данных без категориальных признаков

In [36]:
stats_models["ElasticNet"].params

const               0.634198
popularity          0.000000
duration_ms         0.001367
danceability       -0.022598
loudness            0.128941
speechiness         0.022609
acousticness       -0.105532
instrumentalness    0.026797
liveness            0.023973
valence             0.031885
tempo               0.006123
dtype: float64

In [37]:
stats_models_cat["ElasticNet"].params
model_params[abs(model_params) < 1E-6]

Series([], dtype: float64)

ElasticNet работает более избирательно, нулевые веса (в первом случае) есть, но их меньше чем у Lasso

In [38]:
i, coefs = sklearn_models["ElasticNet"].intercept_, sklearn_models["ElasticNet"].coef_
print(i, coefs)

0.6420033217438598 [-0.          0.00136689 -0.02260264  0.12894748  0.02260992 -0.10552998
  0.02679991  0.02397156  0.03188611  0.00612233]


In [39]:
i, coefs = sklearn_models_cat["ElasticNet"].intercept_, sklearn_models_cat["ElasticNet"].coef_
print(i, coefs[abs(coefs) < 1E-6])

0.6420033217438598 [-0. -0.  0. -0. -0. -0. -0.  0. -0.  0.  0.  0.  0.  0. -0. -0.  0.  0.
 -0.  0.  0. -0. -0. -0. -0.  0.  0.  0. -0. -0.  0.  0. -0.  0. -0. -0.
  0. -0.  0. -0. -0. -0.  0. -0.  0.  0.  0. -0. -0. -0.  0.  0. -0. -0.
 -0.  0. -0. -0. -0. -0.  0. -0. -0. -0.  0. -0.  0.  0. -0.  0.  0. -0.
 -0. -0. -0. -0.  0. -0.  0.  0.  0. -0.  0. -0. -0.]


Аналогичная ситуация у модели из sklearn. В случае датасета с категориальными пременными нулевые веса есть в отличие от statsmodels

#### 5. [1 балл] Реализуйте один из алгоритмов отбора признаков (Elimination by P-value, Forward elimination, Backward elimination), сделайте выводы.

In [40]:
def pvalue_elimination(X, y, significance_level=0.05):
    X = sm.add_constant(X)

    ols = sm.OLS(y, X).fit()
    highest_pvalue = np.max(ols.pvalues)

    while highest_pvalue > significance_level:
      idx = np.argmax(ols.pvalues)
      X = X.drop(X.columns[idx], axis=1)

      ols = sm.OLS(y, X).fit()
      print("R2 score: {r2_score()}")
      highest_pvalue = np.max(ols.pvalues)

    return X.iloc[: , 1:]


In [41]:
print(X_train.columns)
res = pvalue_elimination(X_train, y_train)

Index(['popularity', 'duration_ms', 'danceability', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       ...
       'track_genre_study', 'track_genre_swedish', 'track_genre_synth-pop',
       'track_genre_tango', 'track_genre_techno', 'track_genre_trance',
       'track_genre_trip-hop', 'track_genre_turkish',
       'track_genre_world-music', 'mode_1'],
      dtype='object', length=140)
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}
R2 score: {r2_score()}


In [42]:
res.columns

Index(['popularity', 'duration_ms', 'danceability', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       ...
       'track_genre_study', 'track_genre_swedish', 'track_genre_synth-pop',
       'track_genre_tango', 'track_genre_techno', 'track_genre_trance',
       'track_genre_trip-hop', 'track_genre_turkish',
       'track_genre_world-music', 'mode_1'],
      dtype='object', length=128)

С помощью алгоритма смогли отсеять 12 незначимых признаков

In [43]:
def rmse_score(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

#### 6. [1 балл] Найдите лучший (по RMSE) $\alpha$ для регрессиии Lasso, используя кросс-валидацию на 5 фолдов. Вы должны выбрать значение из промежутка $[10^{-4}, 10^{3}]$.

(Код написан по мотивам кода с семинаров)

In [44]:
alphas = np.logspace(-4, 3, base=10)

searcher = GridSearchCV(Lasso(), [{"alpha": alphas}], scoring=rmse_score, cv=5)
searcher.fit(X_train_scaled, y_train)

best_alpha = searcher.best_params_["alpha"]
print("Best alpha = %.4f" % best_alpha)

Best alpha = 0.0001


## Градиентный спуск

#### 7. [3.5 балла] Имплементируйте  Ridge регрессию для MSE loss, обученную на градиентом спуске.


Все вычисления должны быть векторизованы, а циклы Python можно использовать только для итераций градиентного спуска. В качестве критерия остановки необходимо использовать (одновременно):

* проверка абсолютной нормы разницы весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, заданного параметром `tolerance`);

* достижение максимального количества итераций (например, 10000, заданного параметром `max_iter`).

Вам необходимо выполнить:

* Полный градиентный спуск:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Стохастический градиентный спуск:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ является оценкой градиента по набору объектов, выбранных случайным образом.

* Momentum method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$

Чтобы убедиться, что процесс оптимизации действительно выполняется, мы будем использовать атрибут класса `loss_history`. После вызова метода fit он должен содержать значения функции потерь для всех итераций, начиная с первой (до первого шага по антиградиенту).


Вам нужно инициализировать веса случайным вектором из нормального распределения. Ниже приведен шаблон, который должен содержать код, реализующий все варианты моделей.

In [45]:
from sklearn.base import BaseEstimator

class LinReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum',
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2, reg_cf=0.01, alpha=1e-3, epsilon=1e-6):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        reg_cf: float
            regularization coefficient
        epsilon: float
            numerical stability
        """

        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.reg_cf = reg_cf
        self.eta = eta
        self.loss_history = None # list of loss function values at each training iteration
        self.epsilon = epsilon

    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """
        self.loss_history = []

        self.w = self.w0
        if (self.w0 is None):
            self.w = np.random.normal(size=(X.shape[1]))
        if (self.gd_type == "Momentum"):
            self.calc_momentum(X, y)
        elif (self.gd_type == "Adagrad"):
            self.calc_adagrad(X, y)
        else:
            self.calc_stohastic_gradient_descent(X, y)


        return self

    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')

        return np.dot(X, self.w)

    def calc_gradient(self, X, y):
        """
        X: np.array of shape (l, d) (l can be equal to 1 if stochastic)
        y: np.array of shape (l)
        ---
        output: np.array of shape (d)
        """
        if (self.gd_type != "StochasticDescent"):
            self.delta = 1.0
        n_objects = X.shape[0]
        batch_size = int(self.delta * X.shape[0])
        sample = np.random.randint(n_objects, size=batch_size)
        return 2 * np.dot(X.iloc[sample].T, np.dot(X.iloc[sample], self.w) - y.iloc[sample]) / batch_size + 2 * self.reg_cf * self.w


    def calc_stohastic_gradient_descent(self, X, y):
        for i in range(self.max_iter):
            self.loss_history.append(self.calc_loss(X, y))
            diff = -self.eta * self.calc_gradient(X, y)
            if np.all(np.abs(diff) <= self.tolerance):
                break
            self.w += diff

    def calc_momentum(self, X, y):
        h = 0
        for i in range(self.max_iter):
            self.loss_history.append(self.calc_loss(X, y))
            h = self.alpha * h + self.eta * self.calc_gradient(X, y)
            if np.all(np.abs(h) <= self.tolerance):
                break
            self.w -= h

    def calc_adagrad(self, X, y):
        G = 0
        for i in range(self.max_iter):
            self.loss_history.append(self.calc_loss(X, y))
            gradient = self.calc_gradient(X, y)
            G += gradient**2
            diff = self.eta / np.sqrt(G + self.epsilon) * gradient
            if np.all(np.abs(diff) <= self.tolerance):
                break
            self.w -= diff
            self.loss_history.append(self.calc_loss(X, y))

    def calc_loss(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: float
        """
        return np.mean((np.dot(X, self.w) - y) ** 2) + self.reg_cf * np.linalg.norm(self.w)**2

#### 8. [1 балл] Натренируйте и провалидируйте "ручные" модели на тех же даннных, сравните качество с моделями из Sklearn и StatsModels. Исследуйте влияние параметров `max_iter` и `alpha` на процесс оптимизации. Соответствует ли оно вашим ожиданиям?

In [46]:
y = data_without_categorical["energy"]
X = data_without_categorical.drop("energy", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [47]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

X_train_const = sm.add_constant(X_train_scaled)
X_test_const = sm.add_constant(X_test_scaled)

In [48]:
model = LinReg(gd_type="GradientDescent")
model.fit(X_train_const, y_train)

loss_gd = model.loss_history.copy()
y_pred = model.predict(X_test_const)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}, R2: {r2}")

RMSE: 0.12262910932357185, R2: 0.7606496094219793


In [49]:
model = LinReg(gd_type="StochasticDescent")
model.fit(X_train_const, y_train)

loss_sgd = model.loss_history.copy()

y_pred = model.predict(X_test_const)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}, R2: {r2}")

RMSE: 0.12255574136318846, R2: 0.7609359263801019


In [50]:
model = LinReg(gd_type="Momentum")
model.fit(X_train_const, y_train)

loss_momentum = model.loss_history.copy()

y_pred = model.predict(X_test_const)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}, R2: {r2}")

RMSE: 0.12258779020714711, R2: 0.7608108775020429


In [51]:
model = LinReg(gd_type="Adagrad", eta=1)
model.fit(X_train_const, y_train)

loss_adagrad = model.loss_history.copy()

y_pred = model.predict(X_test_const)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}, R2: {r2}")

RMSE: 0.12203076976940162, R2: 0.7629796176944902


In [52]:
y = data_encoded["energy"]
X = data_encoded.drop("energy", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [53]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

X_train_const = sm.add_constant(X_train_scaled)
X_test_const = sm.add_constant(X_test_scaled)

In [54]:
model = LinReg(gd_type="GradientDescent")
model.fit(X_train_const, y_train)

loss_gd_cat = model.loss_history.copy()

y_pred = model.predict(X_test_const)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}, R2: {r2}")

RMSE: 0.12375616154905388, R2: 0.7562297772853409


In [55]:
model = LinReg(gd_type="StochasticDescent")
model.fit(X_train_const, y_train)

loss_sgd_cat = model.loss_history.copy()

y_pred = model.predict(X_test_const)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}, R2: {r2}")

RMSE: 0.14557553475365323, R2: 0.6626942218145291


In [56]:
model = LinReg(gd_type="Momentum")
model.fit(X_train_const, y_train)

loss_momentum_cat = model.loss_history.copy()

y_pred = model.predict(X_test_const)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}, R2: {r2}")

RMSE: 0.14449629117247398, R2: 0.6676770052928618


In [57]:
model = LinReg(gd_type="Adagrad", eta=1)
model.fit(X_train_const, y_train)

loss_adagraf_cat = model.loss_history.copy()

y_pred = model.predict(X_test_const)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}, R2: {r2}")

RMSE: 0.11246109205326398, R2: 0.7986963796706816


Сравним с моделями из sklearn и sm. Так, метод адаграда очень близок к результату, которые выдавали модели sklearn и sm модели. Остальные методы показали себя хуже, однако это может быть связано с тем, что не были подобраны оптимальные гиперпараметры. В среднем $R^2$ находится на уровне 0.76, что очень близко к моделям из библиотек

Исследуем влияение max_iter и alpha

In [58]:
y = data_without_categorical["energy"]
X = data_without_categorical.drop("energy", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [59]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

X_train_const = sm.add_constant(X_train_scaled)
X_test_const = sm.add_constant(X_test_scaled)

In [66]:
import plotly.graph_objects as go

alphas = np.linspace(10**(-3), 10**(-1), 5)
max_iters = np.arange(1000, 10000, 1000)

alpha_graphics = []
for alpha in alphas:
    model = LinReg(gd_type="Momentum", alpha=alpha)
    model.fit(X_train_const, y_train)
    alpha_graphics.append(go.Scatter(x=np.arange(1, 1001, 1), y=model.loss_history, name=f'alpha = {alpha}'))
    y_pred = model.predict(X_test_const)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"alpha: {alpha} RMSE: {rmse}, R2: {r2}")

mi_graphics = []
for max_iter in max_iters:
    model = LinReg(gd_type="Momentum", max_iter=max_iter)
    model.fit(X_train_const, y_train)
    mi_graphics.append(go.Scatter(x=np.arange(1, 1001, 1), y=model.loss_history, name=f'max_iter = {max_iter}'))

    y_pred = model.predict(X_test_const)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"max_iter: {max_iter} RMSE: {rmse}, R2: {r2}")

alpha: 0.001 RMSE: 0.12275810264012024, R2: 0.7601458001823334
alpha: 0.025750000000000002 RMSE: 0.12288710298069828, R2: 0.7596414337883127
alpha: 0.0505 RMSE: 0.12266819734979245, R2: 0.7604969992388914
alpha: 0.07525000000000001 RMSE: 0.12277972727470032, R2: 0.7600612889989145
alpha: 0.1 RMSE: 0.12243865792439537, R2: 0.7613924871777666
max_iter: 1000 RMSE: 0.12254149969595851, R2: 0.7609914843347809
max_iter: 2000 RMSE: 0.12263443171903037, R2: 0.7606288322168874
max_iter: 3000 RMSE: 0.12254336686615366, R2: 0.7609842007128592
max_iter: 4000 RMSE: 0.12268356698985254, R2: 0.7604369787021328
max_iter: 5000 RMSE: 0.12329440416032617, R2: 0.7580454881774094
max_iter: 6000 RMSE: 0.1226595254136766, R2: 0.7605308610115863
max_iter: 7000 RMSE: 0.12298421902890015, R2: 0.7592613792953926
max_iter: 8000 RMSE: 0.12285679281259909, R2: 0.7597599882973978
max_iter: 9000 RMSE: 0.12306755327381762, R2: 0.7589350192424249


In [67]:
fig = go.Figure(data=alpha_graphics)
fig.update_layout(title='Alpha',
                 xaxis=dict(title="iteration"),
                yaxis=dict(title="loss")
)
fig.show()

In [68]:
fig = go.Figure(data=alpha_graphics)
fig.update_layout(title='max_iter',
                 xaxis=dict(title="iteration"),
                yaxis=dict(title="loss")
)
fig.show()

Эти два параметра с увеличением могут улучшить качество модели, однако сходимость при этом, как правило становится медленее. Поэтому их следует выбирать так, чтобы был баланс между качеством и скоростью сходимости

#### 9. [1 балл] Постройте графики (там же) зависимости значения функции потерь от номера итерации для всех моделей (полного градиентого спуска, стохастического гс, Momentum и Adagrad). Сделайте выводы о скорости сходимости различных модификаций градиентного спуска.


Не забывайте о том, как должен выглядеть *красивый* график!

In [69]:
import plotly.graph_objects as go

full_gd_loss_iter = go.Scatter(x=np.arange(1, 1001, 1), y=loss_gd_cat, name='Full GD')
sgd_loss_iter = go.Scatter(x=np.arange(1, 1001, 1), y=loss_sgd_cat, name='SGD')
momentum_loss_iter = go.Scatter(x=np.arange(1, 1001, 1), y=loss_momentum_cat, name='Momentum')
adagrad_loss_iter = go.Scatter(x=np.arange(1, 1001, 1), y=loss_adagraf_cat, name='Adagrad')

fig = go.Figure(data=[full_gd_loss_iter, sgd_loss_iter, momentum_loss_iter, adagrad_loss_iter])
fig.update_layout(title='Data with categorical',
                 xaxis=dict(title="iteration"),
                yaxis=dict(title="loss")
)
fig.show()

In [70]:
import plotly.graph_objects as go

full_gd_loss_iter = go.Scatter(x=np.arange(1, 1001, 1), y=loss_gd, name='Full GD')
sgd_loss_iter = go.Scatter(x=np.arange(1, 1001, 1), y=loss_sgd, name='SGD')
momentum_loss_iter = go.Scatter(x=np.arange(1, 1001, 1), y=loss_momentum, name='Momentum')
adagrad_loss_iter = go.Scatter(x=np.arange(1, 1001, 1), y=loss_adagrad, name='Adagrad')

fig = go.Figure(data=[full_gd_loss_iter, sgd_loss_iter, momentum_loss_iter, adagrad_loss_iter])
fig.update_layout(title='Data with no categorical',
                 xaxis=dict(title="iteration"),
                yaxis=dict(title="loss")
)
fig.show()

Быстрее всех сходится метод адаграда. Со стандартными параметрами на данных стохастический градиент, полный и метод "моментум" сходятся примерно в одном темпе. Однако их темп сходимости зависит от подобранных гиперпараметров. Так, на последнем графике полный и стохастический градиентные спуски оказываются "быстрее" моментума